In [ ]:
#PRIYANSHU PANCHAL 210968174

In [ ]:
# importing the libraries
import csv
import sys
import time


In [ ]:
# intializing the Node class
class Node():
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action

In [ ]:
# stack class for depth first search
class StackFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[-1]
            self.frontier = self.frontier[:-1]
            return node

In [ ]:
# queue class for breadth first search
class QueueFrontier(StackFrontier):

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node

In [ ]:
# maps names to a set of corresponding person_ids
names = {}

# maps person_ids to a dictionary of: name, birth, movies (a set of movie_ids)
people = {}

# maps movie_ids to a dictionary of: title, year, stars (a set of person_ids)
movies = {}

In [ ]:
# determining the neighbors of passed person ID
def neighbors_for_person(person_id):
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    for movie_id in movie_ids:
        for person_id in movies[movie_id]["stars"]:
            neighbors.add((movie_id, person_id))
    return neighbors

In [ ]:

# resolving name ambiguities
def person_id_for_name(name):
    """
    Returns the IMDB id for a person's name,
    resolving ambiguities as needed.
    """
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]

In [ ]:
#PRIYANSHU PANCHAL 210968174
#finding the shortest path between the source and target
def shortest_path(method, source, target):
    """
    Returns the shortest list of (movie_id, person_id) pairs
    that connect the source to the target.
    If no possible path, returns None.
    """
    solution_found = False
    no_solution = False
    solution = list()

    initial = Node(state=source, parent=None, action=None)

    if method=='bfs':
      frontier = QueueFrontier()
    if method=='dfs':
      frontier = StackFrontier()
    frontier.add(initial)
    explored = set()
    i = 0
    while solution_found == False:

        if frontier.empty() == True:
            no_solution = True
            solution_found = True

        node = frontier.remove()
        # print("\n\nNode in= ",node, ' i=', i)

        if node.state == target:
            # Return the solution
            solution_found = True
            while node.parent is not None:
                pid, mid = node.state, node.action
                solution.append((mid, pid))
                node = node.parent
            solution.reverse()

        explored.add(node)
        children = neighbors_for_person(node.state)
        for child in children:
            child_node = Node(state=child[1], action=child[0],parent=node)
            frontier.add(child_node)
            if child_node.state == target:
                # Return the solution
                solution_found = True
                while child_node.parent is not None:
                    pid, mid = child_node.state, child_node.action
                    solution.append((mid, pid))
                    child_node = child_node.parent
                solution.reverse()

    if solution_found == True:
        if no_solution == True:
            return None
        return solution

In [ ]:
# converting the data to a dictionary
def load_data(folder):
    # Load people
    with open(f"{folder}_people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])

    # Load movies
    with open(f"{folder}_movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # Load stars
    with open(f"{folder}_stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass

In [ ]:
# main function to compile the functions
def main(folder, method):

    # Load data from files into memory
    print("Loading data...")
    load_data(folder)
    print("Data loaded.")

    source = person_id_for_name(input("Enter Name: "))
    if source is None:
        sys.exit("Person not found.")
    target = person_id_for_name(input("Enter Name: "))
    if target is None:
        sys.exit("Person not found.")

    print(str.format('Using {}: ', method))

    path = shortest_path(method, source, target)

    if path is None:
        print("Not connected.")
    else:
        degrees = len(path)
        print(f"{degrees} degrees of separation.")
        path = [(None, source)] + path
        for i in range(degrees):
            person1 = people[path[i][1]]["name"]
            person2 = people[path[i + 1][1]]["name"]
            movie = movies[path[i + 1][0]]["title"]
            print(f"{i + 1}: {person1} and {person2} starred in {movie}")

In [ ]:
start = time.time()
main('small', 'bfs')
end = time.time()
print(str.format('Elapsed time: {} seconds', end-start))

Loading data...
Data loaded.
Enter Name: Tom Hanks
Enter Name: Cary Elwes
Using bfs: 
2 degrees of separation.
1: Tom Hanks and Robin Wright starred in Forrest Gump
2: Robin Wright and Cary Elwes starred in The Princess Bride
Elapsed time: 38.70625615119934 seconds


In [ ]:
start = time.time()
main('small', 'dfs')
end = time.time()
print(str.format('Elapsed time: {} seconds', end-start))

Loading data...
Data loaded.
Enter Name: Tom Hanks
Enter Name: Cary Elwes
Using dfs: 
2 degrees of separation.
1: Tom Hanks and Robin Wright starred in Forrest Gump
2: Robin Wright and Cary Elwes starred in The Princess Bride
Elapsed time: 13.601578712463379 seconds


In [ ]:
#PRIYANSHU PANCHAL 210968174